In [7]:
import csv
import json
import sys
import pandas as pd
import os

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz    
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip3 install -q findspark
!pip3 install -q pyspark
!pip3 install -q kaggle

# Use my own kaggle.json? Or let the user upload it?
from google.colab import files
print("Upload kaggle.json to use kaggle APIs")
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

# !curl -OL https://raw.githubusercontent.com/Bertonc98/ProgettoAMD/main/sample.csv

!kaggle datasets files "ashirwadsangwan/imdb-dataset"
!kaggle datasets download "ashirwadsangwan/imdb-dataset" -f "title.principals.tsv.gz"
!unzip "title.principals.tsv.gz.zip"
!rm "title.principals.tsv.gz.zip"
!gunzip -k "title.principals.tsv.gz"
!rm "title.principals.tsv.gz"

     |████████████████████████████████| 212.3MB 67kB/s 
     |████████████████████████████████| 204kB 19.9MB/s 


Saving kaggle.json to kaggle.json
kaggle.json
name                      size  creationDate         
-----------------------  -----  -------------------  
title.principals.tsv.gz  290MB  2019-11-18 18:36:47  
name.basics.tsv.gz       180MB  2019-11-18 18:36:47  
title.akas.tsv.gz        158MB  2019-11-18 18:36:47  
title.ratings.tsv.gz       5MB  2019-11-18 18:36:47  
title.basics.tsv.gz      108MB  2019-11-18 18:36:47  
 95% 273M/287M [00:03<00:00, 90.0MB/s]
100% 287M/287M [00:03<00:00, 96.0MB/s]
Archive:  title.principals.tsv.gz.zip
  inflating: title.principals.tsv.gz  


In [3]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

home = os.environ["SPARK_HOME"]
os.environ["PYTHONPATH"] = home+"/python"+home+"/python/lib/py4j-0.10.9-src.zip:"+os.environ["PYTHONPATH"]

In [4]:
import findspark
from pyspark.sql import SparkSession
import pyspark
from pyspark import SparkConf
findspark.init()
conf = SparkConf()
#Why 5g?
conf.set("spark.driver.memory", "5g")
conf.set("spark.executor.memory", "5g")
spark = SparkSession.builder.config(conf = conf).master("local[*]").getOrCreate()

In [5]:
sourceFile = 'title.principals.tsv'

data = spark.read.options(delimiter = "\t", header=True).csv(sourceFile)

from pyspark.sql.window import Window

from pyspark.sql.functions import monotonically_increasing_id,row_number

rdd_result = data.select(['nconst', 'category'])\
            .where("category == 'actress' or category == 'actor' ")\
            .select(['nconst'])\
            .distinct()

rdd_result = rdd_result.withColumn("idx",row_number().over(Window.orderBy(monotonically_increasing_id())))


In [10]:
rdd_result.show()

+---------+---+
|   nconst|idx|
+---------+---+
|nm0509806|  1|
|nm0346933|  2|
|nm4021589|  3|
|nm0706926|  4|
|nm0636620|  5|
|nm0116227|  6|
|nm0709856|  7|
|nm0835871|  8|
|nm0183991|  9|
|nm0933355| 10|
|nm0869418| 11|
|nm0107574| 12|
|nm0829397| 13|
|nm0733443| 14|
|nm2423235| 15|
|nm0000847| 16|
|nm0222350| 17|
|nm0213638| 18|
|nm0635515| 19|
|nm0650819| 20|
+---------+---+
only showing top 20 rows



nm0509806|  1|

|nm0346933|  2|

|nm4021589|  3|

|nm0706926|  4|

|nm0636620|  5|

|nm0116227|  6|

|nm0709856|  7|

|nm0835871|  8|

|nm0183991|  9|

|nm0933355| 10|

|nm0869418| 11|

|nm0107574| 12|

|nm0829397| 13|

|nm0733443| 14|

|nm2423235| 15|

|nm0000847| 16|

|nm0222350| 17|

|nm0213638| 18|

|nm0635515| 19|

|nm0650819| 20|

Create preprocessed file

In [11]:
dictionary = {key:value for (key, value) in rdd_result.collect()}


In [ ]:
filter_actors = lambda row: row["category"] in ["actor", "actress"]

map_actors = lambda row: (row["tconst"], dictionary[row["nconst"]]) 

rdd_basketId = data.rdd.filter(filter_actors).map(map_actors).groupByKey().mapValues(list).values()
# res = rdd_basketId.collect()
# with open('preprocessed.csv', 'w', newline='') as f:
#     writer = csv.writer(f, delimiter=";")
#     writer.writerows(res)
# rdd_basketId.saveAsTextFile('preprocesed.csv')
# data = spark.read.options(delimiter = ";", header=True).csv("preprocessed.csv")

In [ ]:
# Does cache() store in memory for every node?
# How to map the algorithm PCY() over each node, instead of every element of each node
rdd_basketId.persist(StorageLevel.MEMORY_AND_DISK)

PythonRDD[37] at RDD at PythonRDD.scala:53

In [ ]:
# rdd_basketId.take(10)
rdd_basketId_flatten = rdd_basketId.flatMap(lambda x: [y for y in x])

rdd_basketId_flatten.map(lambda item: (item, 1)).reduceByKey(lambda a, b: a+b).filter(lambda x: x[1]>30).take(10)

[(673023, 179),
 (1858740, 127),
 (1437683, 253),
 (560898, 65),
 (1512771, 157),
 (1391169, 53),
 (1326299, 85),
 (990444, 50),
 (224393, 45),
 (1727804, 86)]